<a href="https://colab.research.google.com/github/CGS2002/PRECOG-_TASK/blob/main/Bonus_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import numpy as np
from sklearn.cluster import KMeans
from networkx.algorithms import community
from google.colab import drive
drive.mount('/content/drive')
glink='/content/drive/MyDrive/IIIT Graph/' # Change the link to the folder ontaing files

Mounted at /content/drive


In [2]:
dfe=pd.read_csv(f'{glink}Cit-HepPh.txt', sep='\t', header=None, names=['source', 'target'],skiprows=4)
dfn=pd.read_csv(f'{glink}cit-HepPh-dates.txt', sep='\t', header=None, names=['Node', 'Time'],skiprows=1)
dfn['Time']=pd.to_datetime(dfn['Time'])

In [3]:
df = pd.merge(dfe, dfn, left_on='source', right_on='Node', how='inner')
df = pd.merge(df, dfn, left_on='target', right_on='Node', how='inner', suffixes=('_source', '_target'))
df_sorted=df.sort_values('Time_target')
df_sorted['Time_Edge']=df_sorted[['Time_target','Time_source']].max(axis=1)
df_sorted['Year']=df_sorted['Time_Edge'].dt.to_period('Y')


In [149]:
testing = df_sorted[df_sorted['Time_Edge']<='1993-06-24'].copy()
# testing

In [74]:
GT=nx.DiGraph()
GT.add_nodes_from(list(set(testing[['source', 'target']].values.flatten())))
m=[tuple(x[['source', 'target']]) for x in testing.to_records(index=False)]
GT.add_edges_from(m)

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [76]:
G=GT.copy()


In [103]:
label_encoder = LabelEncoder()
label_encoder.fit(list(G.nodes()))

PG=nx.pagerank(G)
def common_in_neighbors(G, node1, node2):
    in_neighbors_node1 = set(G.predecessors(node1))
    in_neighbors_node2 = set(G.predecessors(node2))
    return len(in_neighbors_node1.intersection(in_neighbors_node2))

def common_out_neighbors(G, node1, node2):
    out_neighbors_node1 = set(G.successors(node1))
    out_neighbors_node2 = set(G.successors(node2))
    return len(out_neighbors_node1.intersection(out_neighbors_node2))

def jaccard_coefficient_in_neighbors(G, node1, node2):
    in_neighbors_node1 = set(G.predecessors(node1))
    in_neighbors_node2 = set(G.predecessors(node2))
    if len(in_neighbors_node1.union(in_neighbors_node2)) == 0:
        return 0.0
    return len(in_neighbors_node1.intersection(in_neighbors_node2)) / len(in_neighbors_node1.union(in_neighbors_node2))

def jaccard_coefficient_out_neighbors(G, node1, node2):
    out_neighbors_node1 = set(G.successors(node1))
    out_neighbors_node2 = set(G.successors(node2))
    if len(out_neighbors_node1.union(out_neighbors_node2)) == 0:
        return 0.0
    return len(out_neighbors_node1.intersection(out_neighbors_node2)) / len(out_neighbors_node1.union(out_neighbors_node2))
def betweenness_centrality(G, node):
    return nx.betweenness_centrality(G)[node]

def closeness_centrality(G, node):
    return nx.closeness_centrality(G)[node]

# Extract features for each edge
features = []
labels = []

for edge in G.edges():
    source, target = edge
    features.append([
         label_encoder.transform([source])[0],
        label_encoder.transform([target])[0],
        G.in_degree(source),
        G.out_degree(source),
        G.in_degree(target),
        G.out_degree(target),
        common_in_neighbors(G, source, target),
        common_out_neighbors(G, source, target),
        jaccard_coefficient_in_neighbors(G, source, target),
        jaccard_coefficient_out_neighbors(G, source, target),
          nx.clustering(G,source),
        nx.clustering(G,target),
        PG[source],
        PG[target]

    ])
    labels.append(1)
for edge in nx.non_edges(G):
    source, target = edge
    features.append([
         label_encoder.transform([source])[0],
        label_encoder.transform([target])[0],
        G.in_degree(source),
        G.out_degree(source),
        G.in_degree(target),
        G.out_degree(target),
        common_in_neighbors(G, source, target),
        common_out_neighbors(G, source, target),
        jaccard_coefficient_in_neighbors(G, source, target),
        jaccard_coefficient_out_neighbors(G, source, target),
        nx.clustering(G,source),
        nx.clustering(G,target),
        PG[source],
        PG[target]

    ])
    labels.append(0)

features = np.array(features)
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report_output)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9978461538461538
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     77835
           1       0.20      0.01      0.01       165

    accuracy                           1.00     78000
   macro avg       0.60      0.50      0.51     78000
weighted avg       1.00      1.00      1.00     78000



In [145]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.9977948717948718
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     77835
           1       0.43      0.13      0.20       165

    accuracy                           1.00     78000
   macro avg       0.71      0.56      0.60     78000
weighted avg       1.00      1.00      1.00     78000



In [146]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.998
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     77835
           1       0.67      0.11      0.19       165

    accuracy                           1.00     78000
   macro avg       0.83      0.55      0.59     78000
weighted avg       1.00      1.00      1.00     78000



In [137]:
edge_to_remove = list(G.edges())[752]
source_node, target_node = edge_to_remove
removed_edge_features = [
         label_encoder.transform([source])[0],
        label_encoder.transform([target])[0],
        G.in_degree(source_node),
        G.out_degree(source_node),
        G.in_degree(target_node),
        G.out_degree(target_node),
        common_in_neighbors(G, source_node, target_node),
        common_out_neighbors(G, source_node, target_node),
        jaccard_coefficient_in_neighbors(G, source_node, target_node),
        jaccard_coefficient_out_neighbors(G, source_node, target_node),
         nx.clustering(G,source_node),
        nx.clustering(G,target_node),
        PG[source],
        PG[target]
    ]
removed_edge_features = np.array(removed_edge_features).reshape(1, -1)

prediction = model.predict(removed_edge_features)

print(f"Actual Edge Exists: {G.has_edge(source_node, target_node)}")
print(f"Predicted Edge Exists: {prediction[0]}")


Actual Edge Exists: True
Predicted Edge Exists: 1


In [147]:
prediction = clf.predict(removed_edge_features)
prediction

array([0])

In [148]:
prediction = model_xgb.predict(removed_edge_features)
prediction

array([0])

Sources

https://www.irjet.net/archives/V7/i11/IRJET-V7I1120.pdf
